In [1]:
import zipfile
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

!pip install pydicom
import pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###### сравним размерности датасетов
получим списки:
- пациентов по которым есть маски
- пациентов по которым масок нет

In [ ]:
# подгрузим данные стрезов по пациентом (сформированны в блокнотах Datasets dcm.ipynb, Datasets nii.ipynb)

count_slice_dcm = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/count_slice_dcm.csv').rename(columns ={'slice': 'slice_dcm'})
count_slice_gz = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/count_slice_gz.csv').rename(columns ={'slice': 'slice_gz'})

count_slice = pd.merge(count_slice_dcm, count_slice_gz, on = 'name_pictures', how = 'left')

In [ ]:
# сравним кол-во срезов по картинкам по поциентам

count_slice['mask'] = np.where (count_slice['slice_gz'] == count_slice['slice_gz'], 'yes', 'no')
count_slice['dcm=gz'] = np.where((count_slice['mask'] == 'yes')&(count_slice['slice_dcm'] == count_slice['slice_gz']), True, None)
count_slice['dcm=gz'] = np.where((count_slice['mask'] == 'yes')&(count_slice['slice_dcm'] != count_slice['slice_gz']), False, count_slice['dcm=gz'] )
count_slice['dcm=gz'].value_counts()

True    77
Name: dcm=gz, dtype: int64

In [ ]:
# посмотрим кол-во пациентов по размерностям срезов
count_slice['slice_gz'].value_counts()

134.0    23
94.0      7
135.0     3
90.0      3
133.0     2
93.0      2
110.0     2
114.0     2
176.0     2
118.0     2
101.0     2
131.0     2
91.0      2
184.0     1
108.0     1
92.0      1
97.0      1
136.0     1
100.0     1
119.0     1
96.0      1
130.0     1
116.0     1
106.0     1
111.0     1
297.0     1
123.0     1
105.0     1
125.0     1
104.0     1
89.0      1
99.0      1
102.0     1
95.0      1
82.0      1
132.0     1
Name: slice_gz, dtype: int64

In [ ]:
# получим списки пациентов с масками и нет

patient_mask = count_slice[count_slice['mask'] == 'yes']
patient_no_mask = count_slice[count_slice['mask'] != 'yes']

In [ ]:
patient_mask['name_pictures'].unique()

array(['LUNG1-001', 'LUNG1-002', 'LUNG1-005', 'LUNG1-008', 'LUNG1-013',
       'LUNG1-016', 'LUNG1-018', 'LUNG1-024', 'LUNG1-026', 'LUNG1-028',
       'LUNG1-035', 'LUNG1-038', 'LUNG1-042', 'LUNG1-046', 'LUNG1-050',
       'LUNG1-051', 'LUNG1-053', 'LUNG1-060', 'LUNG1-065', 'LUNG1-068',
       'LUNG1-088', 'LUNG1-091', 'LUNG1-093', 'LUNG1-097', 'LUNG1-104',
       'LUNG1-107', 'LUNG1-113', 'LUNG1-115', 'LUNG1-117', 'LUNG1-128',
       'LUNG1-133', 'LUNG1-143', 'LUNG1-148', 'LUNG1-149', 'LUNG1-156',
       'LUNG1-168', 'LUNG1-170', 'LUNG1-177', 'LUNG1-186', 'LUNG1-195',
       'LUNG1-196', 'LUNG1-205', 'LUNG1-229', 'LUNG1-242', 'LUNG1-249',
       'LUNG1-252', 'LUNG1-253', 'LUNG1-277', 'LUNG1-285', 'LUNG1-286',
       'LUNG1-291', 'LUNG1-296', 'LUNG1-299', 'LUNG1-301', 'LUNG1-303',
       'LUNG1-307', 'LUNG1-312', 'LUNG1-314', 'LUNG1-320', 'LUNG1-325',
       'LUNG1-331', 'LUNG1-336', 'LUNG1-340', 'LUNG1-343', 'LUNG1-348',
       'LUNG1-361', 'LUNG1-362', 'LUNG1-366', 'LUNG1-367', 'LUNG

In [ ]:
patient_no_mask['name_pictures'].unique()

array(['LUNG1-004', 'LUNG1-006', 'LUNG1-007', 'LUNG1-009', 'LUNG1-010',
       'LUNG1-011', 'LUNG1-012', 'LUNG1-015', 'LUNG1-017', 'LUNG1-019',
       'LUNG1-020', 'LUNG1-022', 'LUNG1-023', 'LUNG1-025', 'LUNG1-027',
       'LUNG1-029', 'LUNG1-030', 'LUNG1-032', 'LUNG1-033', 'LUNG1-034',
       'LUNG1-036', 'LUNG1-037', 'LUNG1-039', 'LUNG1-040', 'LUNG1-041',
       'LUNG1-043', 'LUNG1-044', 'LUNG1-045', 'LUNG1-047', 'LUNG1-048',
       'LUNG1-049', 'LUNG1-052', 'LUNG1-054', 'LUNG1-055', 'LUNG1-056',
       'LUNG1-057', 'LUNG1-059', 'LUNG1-062', 'LUNG1-063', 'LUNG1-064',
       'LUNG1-066', 'LUNG1-067', 'LUNG1-070', 'LUNG1-071', 'LUNG1-072',
       'LUNG1-073', 'LUNG1-075', 'LUNG1-076', 'LUNG1-077', 'LUNG1-078',
       'LUNG1-079', 'LUNG1-080', 'LUNG1-081', 'LUNG1-082', 'LUNG1-084',
       'LUNG1-086', 'LUNG1-087', 'LUNG1-089', 'LUNG1-090', 'LUNG1-092',
       'LUNG1-094', 'LUNG1-096', 'LUNG1-098', 'LUNG1-099', 'LUNG1-100',
       'LUNG1-101', 'LUNG1-102', 'LUNG1-103', 'LUNG1-105', 'LUNG

# сформируем выборки для теста и обучения

In [3]:
# функция удаляет все пары ключ-значение, ключ которых не начинается с какого-либо значения списка
# нужна чтобы отобрать нужное кол-во примеров

def keys_to_remove(my_dict, key_parts):
  keys_to_remove = []
  for key in my_dict.keys():
      if not any(key.startswith(part) for part in key_parts):
          keys_to_remove.append(key)

  my_dict_new = copy.deepcopy(my_dict)
  for key in keys_to_remove:
      del my_dict_new[key]

  return my_dict_new

In [4]:
# Функция заменяет все значения np.array на 0 - для картинок без выпота

def replace_array_with_zeros(arr):
  for k in list(arr.keys()):
    print(k)
    arr[k] = np.zeros(shape=(512, 512))
    return arr

In [5]:
# функция подгоотавливает картинку на вход модели
from skimage.transform import resize

def  norm_array(array, norm = 'no', rsize = 'no', xsize = 512, ex_dims = 'no'):
    for i in array.keys():       
       
        if norm == 'yes':  # Нормализация значений пикселей к диапазону от 0 до 1
            array[i] = array[i] / np.max(array[i])

        if rsize == 'yes': # Изменение размера изображения до требуемого размера (например, 256x256)
            array[i]= resize(array[i], (xsize, xsize), anti_aliasing=True)

        if ex_dims == 'yes': # Добавление размерности к массиву пикселей для соответствия входному формату нейронной сети
            array[i] = np.expand_dims(array[i], axis=-1)

        # Подготовка изображения для входа в нейронную сеть
        array[i] = array[i].astype(np.float32)

    return array


In [6]:
def x_train_test(dict_images):
    x_val = []                            # Cписок под проверочную выборку

    for img in list(dict_images.keys()):                # Для всех изображений выборки:
        x = image.img_to_array(dict_images[img])       # Перевод изображения в numpy-массив формы: высота x ширина x количество каналов
        x_val.append(x)                   # Добавление элемента в x_train

    x_val = np.array(x_val)               # Перевод всей выборки в numpy
    print(x_val.shape)                    # Форма x_train

    return x_val

dcm_images

In [ ]:
# для выбора примеров с выпотами:
    # 'LUNG1-001', 'LUNG1-002', 'LUNG1-005', 'LUNG1-008', 'LUNG1-013',
    # 'LUNG1-016', 'LUNG1-018', 'LUNG1-024', 'LUNG1-026', 'LUNG1-028',
    # 'LUNG1-035', 'LUNG1-038', 'LUNG1-042', 'LUNG1-046', 'LUNG1-050',

# для выбора примеров без выпотов:
      #  'LUNG1-004', 'LUNG1-006', 'LUNG1-007', 'LUNG1-009', 'LUNG1-010',
      #  'LUNG1-011', 'LUNG1-012', 'LUNG1-015', 'LUNG1-017', 'LUNG1-019',
      #  'LUNG1-020', 'LUNG1-022', 'LUNG1-023', 'LUNG1-025', 'LUNG1-027',
      #  'LUNG1-029', 'LUNG1-030', 'LUNG1-032', 'LUNG1-033', 'LUNG1-034',
      #  'LUNG1-036', 'LUNG1-037', 'LUNG1-039', 'LUNG1-040', 'LUNG1-041',
      #  'LUNG1-043', 'LUNG1-044', 'LUNG1-045', 'LUNG1-047', 'LUNG1-048',
      #  'LUNG1-049', 'LUNG1-052', 'LUNG1-054', 'LUNG1-055', 'LUNG1-056',
      #  'LUNG1-057', 'LUNG1-059', 'LUNG1-062', 'LUNG1-063', 'LUNG1-064',

In [7]:
effusion = ['LUNG1-001', 'LUNG1-002', 
            'LUNG1-005', 'LUNG1-008'
            ]
no_effusion = ['LUNG1-004', 'LUNG1-006', 
               'LUNG1-007', 'LUNG1-009'
               ]

In [8]:
sl = 2
xsize = 128

In [9]:
print(effusion[:sl] + no_effusion[:sl])
print(effusion[sl:] + no_effusion[sl:])

['LUNG1-001', 'LUNG1-002', 'LUNG1-004', 'LUNG1-006']
['LUNG1-005', 'LUNG1-008', 'LUNG1-007', 'LUNG1-009']


In [10]:
import pickle

# значения с выпотом
with open('/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/dcm_images_data_2d_all_mask_512_1.pickle', 'rb') as handle:
    input_array_1 = pickle.load(handle)
handle.close()

# значения без выпотов
with open('/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/dcm_images_data_2d_all_mask_512_2.pickle', 'rb') as handle:
    input_array_2 = pickle.load(handle)
handle.close()


In [11]:
# собираем одинаковое кол-во примеров с выпотами и нет в один файл

input_array_1 = keys_to_remove(input_array_1, effusion) # значения с выпотом
input_array_2 = keys_to_remove(input_array_2, no_effusion) # значения без выпотов

input_array = input_array_1 | input_array_2

del input_array_1
del input_array_2

In [12]:
# делим примеры на 2 выборки

train_input = keys_to_remove(input_array, effusion[:sl] + no_effusion[:sl])
test_input = keys_to_remove(input_array, effusion[sl:] + no_effusion[sl:])

del input_array


In [13]:
# приводим данные к нужному виду для подачи в модель

train_input = norm_array(train_input, norm = 'yes', rsize = 'yes', xsize = xsize, ex_dims = 'yes')
test_input = norm_array(test_input, norm = 'yes', rsize = 'yes', xsize = xsize, ex_dims = 'yes')

nii_images

In [14]:
import pickle

In [15]:


with open('/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/nii_images_data_2d_all_mask_512_1.pickle', 'rb') as outfile:
    output_array = pickle.load(outfile)
outfile.close()
output_array = keys_to_remove(output_array, effusion)



In [16]:
with open('/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/dcm_images_data_2d_all_mask_512_2.pickle', 'rb') as handle:
    input_array_2 = pickle.load(handle)
handle.close()
input_array_2 = keys_to_remove(input_array_2, no_effusion)


In [17]:
# собираем одинаковое кол-во примеров масок с выпотами и нет в один файл

# output_array = keys_to_remove(output_array, effusion)
# input_array_2 = keys_to_remove(input_array_2, no_effusion)

# заполняем маски примеров отсутствия выпотов 0
for k in list(input_array_2.keys()):
    input_array_2[k] = np.zeros(shape=(512, 512))

# собираем все примерв в один файл
output_array = output_array | input_array_2

del input_array_2

In [18]:
# делим примеры на 2 выборки

train_output = keys_to_remove(output_array, effusion[:sl] + no_effusion[:sl])
test_output = keys_to_remove(output_array, effusion[sl:] + no_effusion[sl:])

del output_array

In [19]:
# приводим данные к нужному виду для подачи в модель

train_output = norm_array(train_output, norm = 'no', rsize = 'yes', xsize = xsize, ex_dims = 'yes')
test_output = norm_array(test_output, norm = 'no', rsize = 'yes', xsize = xsize, ex_dims = 'yes')

# строим\обучаем модель

In [20]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, BatchNormalization, Activation
from tensorflow.keras.preprocessing import image

In [21]:
# # функция пребразования выборок
train_input = x_train_test(train_input)
test_input = x_train_test(test_input)
train_output = x_train_test(train_output)
test_output = x_train_test(test_output)


(473, 128, 128, 1)
(439, 128, 128, 1)
(473, 128, 128, 1)
(439, 128, 128, 1)


In [22]:
def unet2d(input_size=(xsize, xsize, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

model = unet2d()


In [23]:
import matplotlib
from matplotlib import pylab as plt
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import Model 
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import metrics
import gc;
from google.colab import drive
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import BinaryIoU, IoU
from tensorflow.keras.optimizers import Adam
#Для сохранения и загрузки модели с кастомными метриками
from keras.utils import get_custom_objects
import random
import pydicom as dicom
import nibabel as nib
import copy

In [24]:
# from keras.utils.generic_utils import get_custom_objects
# from tensorflow.keras.metrics import BinaryIoU, IoU

def dice_coef(y_true, y_pred, shape=1e-6):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  # print(f'y_true= {type(y_true)} y_pred= {type(y_pred)}')
  intersection = K.sum(y_true_f * y_pred_f)
  calc1=(2. * intersection + shape)
  calc2=  (K.sum(y_true_f) + K.sum(y_pred_f) + shape)
  res=calc1 / calc2
  return  res

def dice_coef_loss(y_true, y_pred):
  return 1 - dice_coef(y_true, y_pred)
# get_custom_objects().update({"dice_coef": dice_coef, "dice_coef_loss": dice_coef_loss})




In [25]:
from keras.callbacks import ModelCheckpoint

# Создаем колбек для сохранения весов модели после каждой эпохи (пусть будет)
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/128weights.{epoch:02d}.h5')


In [26]:
model.compile(optimizer='adam',
                  loss=dice_coef_loss,
                  #loss=loss_combine,
                  metrics=[BinaryIoU(target_class_ids=[0, 1], threshold=0.9,name='binary_io_u')])

In [ ]:
model.fit(train_input, train_output, validation_data=(test_input, test_output), batch_size=5, epochs=20 , 
          callbacks=[checkpoint])

Epoch 1/20
95/95 [==============================] - 1287s 13s/step - loss: 0.7780 - binary_io_u: 0.5000 - val_loss: 0.1936 - val_binary_io_u: 0.5000
Epoch 2/20
 8/95 [=>............................] - ETA: 14:46 - loss: 0.7500 - binary_io_u: 0.5000

##### размер картинки 2д (128*128) 

In [ ]:
model.fit(train_input, train_output, validation_data=(test_input, test_output), batch_size=25, epochs=20 , 
          callbacks=[checkpoint])

Epoch 1/20
19/19 [==============================] - 1199s 64s/step - loss: 0.9984 - binary_io_u: 0.0035 - val_loss: 0.9994 - val_binary_io_u: 0.0034
Epoch 2/20
19/19 [==============================] - 1213s 65s/step - loss: 0.9984 - binary_io_u: 0.0035 - val_loss: 0.9994 - val_binary_io_u: 0.0034
Epoch 3/20
19/19 [==============================] - 1219s 65s/step - loss: 0.9984 - binary_io_u: 0.0035 - val_loss: 0.9994 - val_binary_io_u: 0.0034
Epoch 4/20
19/19 [==============================] - 1178s 63s/step - loss: 0.9984 - binary_io_u: 0.0035 - val_loss: 0.9994 - val_binary_io_u: 0.0034
Epoch 5/20
11/19 [================>.............] - ETA: 6:42 - loss: 0.9987 - binary_io_u: 0.0035

##### размер картинки 2д (256*256) 
-loss: 0.9984 - binary_io_u: 0.4998 - val_loss: 0.9994 - val_binary_io_u: 0.5000

In [ ]:
# checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Colab Notebooks/СТАЖИРОВКА1/256weights.{epoch:02d}.h5')

model.fit(train_input, train_output, validation_data=(test_input, test_output), batch_size=5, epochs=20 , 
          callbacks=[checkpoint])

Epoch 1/20
95/95 [==============================] - 4650s 49s/step - loss: 0.9984 - binary_io_u: 0.4998 - val_loss: 0.9994 - val_binary_io_u: 0.5000
Epoch 2/20
95/95 [==============================] - 4562s 48s/step - loss: 0.9984 - binary_io_u: 0.4998 - val_loss: 0.9994 - val_binary_io_u: 0.5000
Epoch 3/20
95/95 [==============================] - 4519s 48s/step - loss: 0.9984 - binary_io_u: 0.4998 - val_loss: 0.9994 - val_binary_io_u: 0.5000
Epoch 4/20
95/95 [==============================] - 4566s 48s/step - loss: 0.9984 - binary_io_u: 0.4998 - val_loss: 0.9994 - val_binary_io_u: 0.5000
Epoch 5/20
95/95 [==============================] - 4528s 48s/step - loss: 0.9984 - binary_io_u: 0.4998 - val_loss: 0.9994 - val_binary_io_u: 0.5000
Epoch 6/20
67/95 [====================>.........] - ETA: 17:42 - loss: 0.9984 - binary_io_u: 0.4998

KeyboardInterrupt: ignored